In [2]:
from robotic import ry
import numpy as np
import time
from spatial_bicycle_models import BicycleModel

###########################################PART-1 STARTS HERE##########################################

In [3]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('maze.g')

qHome = C.getJointState()
C.setJointState(qHome)

Get Final Pose with KOMO

In [5]:
komo = ry.KOMO(C, 1, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])

ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")


{ time: 0.010529, evals: 54, done: 1, feasible: 0, sos: 29.5425, f: 0, ineq: 0, eq: 5.5 }
size of path: (1, 3)


0

MPC

In [9]:
# Instantiate motion model
car = BicycleModel(length=0.30, width=0.20, reference_path=komo.getPath(), Ts=0.05)


# Define cost matrices and constraints
 ##############
# Controller #
##############

N = 30
Q = sparse.diags([1.0, 0.0, 0.0])
R = sparse.diags([0.5, 0.0])
QN = sparse.diags([1.0, 0.0, 0.0])

v_max = 1.0  # m/s
delta_max = 0.66  # rad
ay_max = 4.0  # m/s^2

# Instantiate MPC controller
mpc_controller = MPC(model=car, N=N, Q=Q,
                    R=R, QN=QN,
                    StateConstraints=v_max,
                    InputConstraints=delta_max,
                    ay_max=ay_max)

# Loop over time steps
for _ in range(100):
    # Get control signals from MPC controller
    control_signals = mpc_controller.get_control()

    # Apply control_signals to your system

    # Optionally, visualize the predicted trajectory
    mpc_controller.show_prediction()


AttributeError: 'numpy.ndarray' object has no attribute 'waypoints'

Trajectory Optimization via KOMO

In [6]:
C.setJointState(qHome)
komo2 = ry.KOMO(C, 1, path.shape[0], 2, True)
komo2.addControlObjective([], 0, 1e-1)
komo2.addControlObjective([], 2, 1e0)
komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])
komo2.initWithPath_qOrg(path)

ret = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
q = komo2.getPath()
print('last path:', q)

# display the path for part 1
for i in range(0, q.shape[0]):
    C.setJointState(q[i])
    C.view(False, f'waypoints{i}')
    time.sleep(0.02)


last path: [[ 3.56821785e-05 -7.26208863e-04 -4.48713801e-06]
 [ 1.09649042e-04 -2.16855756e-03 -1.34446508e-05]
 [ 2.24503096e-04 -4.31697705e-03 -2.68557751e-05]
 ...
 [-4.12301230e+00 -3.20683646e+00  6.60080594e-01]
 [-4.15873110e+00 -3.22874231e+00  6.63031947e-01]
 [-4.19446114e+00 -3.25063894e+00  6.65983299e-01]]


###########################################PART-1 ENDS HERE##########################################